In [2]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from tensorflow import keras
from tensorflow import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
# from keras.layers.core import Dropout, Lambda
# from tensorflow.keras.layers.convolutional import Conv2D, Conv2DTranspose
# from tensorflow.keras.layers.pooling import MaxPooling2D
# from tensorflow.keras.layers.merge import concatenate
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
import os
import sys
import random
import warnings

from PIL import Image
from skimage import data, io, filters
import numpy as np
import pandas as pd
from scipy import ndimage
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import manifold, datasets

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from pylab import imread,subplot,imshow,show
from skimage.transform import resize
from skimage.morphology import label
from tensorflow.keras.preprocessing import image
import cv2

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *


# import keras
# from keras import losses
# from keras import backend as K
# from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.utils import to_categorical
# from keras.layers import Conv2D
# from keras.layers import Conv1D
# from keras.layers import MaxPooling2D
# from keras.layers import Flatten
# from keras.layers import concatenate
# from keras.layers import Conv2DTranspose
# from keras import optimizers
# from keras.layers import Conv1D
# from keras.optimizers import SGD #Stochastic Gradient Descent Optimize
# from keras.utils.np_utils import to_categorical
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# from keras.layers import GlobalAveragePooling2D, Input, multiply, LocallyConnected2D, Lambda, ReLU
# from keras.models import Model, load_model
# from keras.layers import BatchNormalization
# from keras.utils.np_utils import to_categorical
# from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
# from keras.optimizers import Adam

# import keras
import tensorflow as tf


import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

# from keras.models import Model, load_model
# from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
# from keras.layers.core import Lambda, RepeatVector, Reshape
# from keras.layers.convolutional import Conv2D, Conv2DTranspose
# from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
# from keras.layers.merge import concatenate, add
# from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from keras.optimizers import Adam
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.python.platform import build_info as tf_build_info
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout
import imageio
from sklearn.metrics import *
from glob import glob

# Check for path existance if not make a new dir
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
# load dataseet
def load_data(path):
    #"""X = Images and Y=masks""" #true for testing and training
    train_all = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
    train_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))
    train_all = np.random.permutation(train_all)
    number_for_train = 10
    train_x = train_all[:number_for_train]
#     print(train_x)
    train_y = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
    test_x = train_all[number_for_train:]
#     print(test_x)
    test_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))
    return (train_x, train_y), (test_x, test_y)



def augment_data(images, masks, save_path, augment=True):
    H = 512
    W = 512
    
    for idx, (x,y) in tqdm(enumerate(zip(images,masks)), total = len(images)):
        name = x.split ("\\") [- 1] .split (".") [0]
#         print(name)
        print(x)
        # reading image and mask
        x_aaa = x
        x = cv2.imread(x,cv2.IMREAD_COLOR)
        x_aaa = x_aaa.replace('images','1st_manual')
        aaa = x_aaa.replace('tif','gif')
        print(aaa)
        y = imageio.mimread(aaa)[0]
        print(x.shape, y.shape)
        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]

            aug = ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            aug = GridDistortion(p=1)
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = augmented['mask']

            aug = OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = augmented['mask']

            X = [x, x1, x2, x3, x4, x5]
            Y = [y, y1, y2, y3, y4, y5]

        else:
            X = [x]
            Y = [y]
            
        index = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(X) == 1:
                tmp_image_name = f"{name}.jpg"
                tmp_mask_name = f"{name}.jpg"
            else:
                tmp_image_name = f"{name}_{index}.jpg"
                tmp_mask_name = f"{name}_{index}.jpg"

            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1
        
        

if __name__=="__main__":
    seed = 42
    random.seed = seed
    np.random.seed = seed
     
    # load data 
    data_path = 'C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig'
    (train_x, train_y), (test_x, test_y) = load_data(data_path)
    print(f"Train:{len(train_x)}-{len(train_y)}")
    print(f"Test:{len(test_x)}-{len(test_y)}")
    # create directories
    create_dir("new_data/train/image")
    create_dir("new_data/train/mask")
    create_dir("new_data/test/image")
    create_dir("new_data/test/mask")
    
    #augment_data(train_x, train_y, "new_data/train/", augment=True)
    augment_data(train_x, train_y, "new_data/train/", augment=True)
    augment_data(test_x, test_y, "new_data/test/", augment=False)

Train:10-20
Test:10-20


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\images\im0163.tif
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\1st_manual\im0163.gif
(605, 700, 3) (605, 700)


 10%|████████▎                                                                          | 1/10 [00:00<00:02,  3.81it/s]

C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\images\im0082.tif
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\1st_manual\im0082.gif
(605, 700, 3) (605, 700)


...

 40%|█████████████████████████████████▏                                                 | 4/10 [00:00<00:00, 30.39it/s]

C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\images\im0005.tif
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\1st_manual\im0005.gif
(605, 700, 3) (605, 700)
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\images\im0001.tif
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\1st_manual\im0001.gif
(605, 700, 3) (605, 700)
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\images\im0319.tif
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\1st_manual\im0319.gif
(605, 700, 3) (605, 700)
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\images\im0235.tif
C:/Users/aqsas/OneDrive/Documen

 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [00:00<00:00, 28.82it/s]

C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\images\im0162.tif
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\1st_manual\im0162.gif
(605, 700, 3) (605, 700)
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\images\im0003.tif
C:/Users/aqsas/OneDrive/Documents/College stuff/ECE_599_Thesis/datasets/R2UNet/New_data_10_10/orig\training\1st_manual\im0003.gif
(605, 700, 3) (605, 700)


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 29.13it/s]


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1.
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = K.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (K.sum(y_true) + K.sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

# RCNN
def Rec_conv2d_bn(x, nb_filter, nb_row, nb_col,border_mode='same',strides=(1, 1),name=None):
    '''Utility function to apply conv + BN.
    '''
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
        
    #if K.image_dim_ordering() == 'tf':
    bn_axis = 3
    #kernal_initializer = tf.keras.initializers.HeNormal()
    
    x1 = tf.keras.layers.Conv2D(nb_filter, (nb_row, nb_col),strides=strides,
                      padding=border_mode,
                      name=conv_name,kernel_regularizer=tf.keras.regularizers.l2(0.0002),data_format='channels_last')(x)
    x1=tf.keras.layers.Activation('relu')(x1)   
    x1 = tf.keras.layers.BatchNormalization()(x1)
    
    x2 = tf.keras.layers.Conv2D(nb_filter, (nb_row, nb_col),strides=strides,
                      padding=border_mode,
                      name=conv_name,kernel_regularizer=tf.keras.regularizers.l2(0.0002),data_format='channels_last')(x1)              
    x2=tf.keras.layers.Activation('relu')(x2)  
    x2 = tf.keras.layers.BatchNormalization()(x2)

    x12 = tf.keras.layers.add([x1, x2])
    x12 = tf.keras.layers.BatchNormalization()(x12)
    
    x3 = tf.keras.layers.Conv2D(nb_filter, (nb_row, nb_col),strides=strides,
                      padding=border_mode,
                      name=conv_name,kernel_regularizer=tf.keras.regularizers.l2(0.0002),data_format='channels_last')(x12) 
    x3=tf.keras.layers.Activation('relu')(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)

    x13 = tf.keras.layers.add([x1, x3])

    x4 = tf.keras.layers.Conv2D(nb_filter, (nb_row, nb_col),strides=strides,
                      padding='same',
                      name=conv_name,kernel_regularizer=tf.keras.regularizers.l2(0.0002),data_format='channels_last')(x13)
    x4=tf.keras.layers.Activation('relu')(x4) 
    x4 = tf.keras.layers.BatchNormalization()(x4)

   
    x14 = tf.keras.layers.add([x1, x4])
    
   # x5 = tf.keras.layers.Conv2D(nb_filter, (nb_row, nb_col),strides=strides,
                    #  padding='same',
                    #  name=conv_name,kernel_regularizer=tf.keras.regularizers.l2(0.0002),data_format='channels_last')(x14)
   # x5=tf.keras.layers.Activation('relu')(x5) 
  #  x5 = tf.keras.layers.BatchNormalization()(x5)

   
   # x15 = tf.keras.layers.add([x1, x5])
    
   # x6 = tf.keras.layers.Conv2D(nb_filter, (nb_row, nb_col),strides=strides,
                     # padding='same',
                     # name=conv_name,kernel_regularizer=tf.keras.regularizers.l2(0.0002),data_format='channels_last')(x15)
    #x6=tf.keras.layers.Activation('relu')(x5) 
    #x6 = tf.keras.layers.BatchNormalization()(x5)
    #x16 = tf.keras.layers.add([x1, x6])
    
    #x7 = tf.keras.layers.Conv2D(nb_filter, (nb_row, nb_col),strides=strides,
     #                 padding='same',
      #                name=conv_name,kernel_regularizer=tf.keras.regularizers.l2(0.0002),data_format='channels_last')(x16)
    #x7=tf.keras.layers.Activation('relu')(x6) 
    #x7 = tf.keras.layers.BatchNormalization()(x6)   
    #x17 = tf.keras.layers.add([x1, x7])
    
   # x8 = tf.keras.layers.Conv2D(nb_filter, (nb_row, nb_col),strides=strides,
                      #padding='same',
                     # name=conv_name,kernel_regularizer=tf.keras.regularizers.l2(0.0002),data_format='channels_last')(x17)
    #x8=tf.keras.layers.Activation('relu')(x7) 
    #x8 = tf.keras.layers.BatchNormalization()(x7)   
    #x18 = tf.keras.layers.add([x1, x8])
    #x = Dropout(0.2)(x18)
    x=tf.keras.layers.Activation('relu')(x14) 
    x = BatchNormalization(axis=bn_axis, name=bn_name)(x)
    return x

def build_R2UNetED(input_shape):
      

    channel_axis = 3
    inputs =  tf.keras.Input(input_shape) 

    # first block
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(inputs)    
    rcnn_bn1 = Rec_conv2d_bn(x, 32, 3, 3)    
    conv1_f = tf.keras.layers.add([x, rcnn_bn1])

    # downsampling first block..
    pool1 = tf.keras.layers.MaxPooling2D((2, 2),data_format='channels_last')(conv1_f)
    conv_pool1 = tf.keras.layers.Conv2D(64, (1, 1), activation='relu', padding='same',data_format='channels_last')(pool1)
    
    # second RRCNN layer...
    rcnn_bn2 = Rec_conv2d_bn(conv_pool1, 64, 3, 3)    
    conv2_f = tf.keras.layers.add([conv_pool1, rcnn_bn2])
    
    # downsampling first block..
    pool2 = tf.keras.layers.MaxPooling2D((2, 2),data_format='channels_last')(conv2_f)
    conv_pool2 = tf.keras.layers.Conv2D(128, (1, 1), activation='relu', padding='same',data_format='channels_last')(pool2)
    
    # third RRCNN layer...
    rcnn_bn3 = Rec_conv2d_bn(conv_pool2, 128, 3, 3)    
    conv3_f = tf.keras.layers.add([conv_pool2, rcnn_bn3])
    
    # downsampling first block..
    pool3 = tf.keras.layers.MaxPooling2D((2, 2),data_format='channels_last')(conv3_f)
    conv_pool3 = tf.keras.layers.Conv2D(256, (1, 1), activation='relu', padding='same',data_format='channels_last')(pool3)
    
    # fourth RRCNN layer...
    rcnn_bn4 = Rec_conv2d_bn(conv_pool3, 256, 3, 3)    
    conv4_f = tf.keras.layers.add([conv_pool3, rcnn_bn4])
      
    # Decoder...
    
    up7_1 = tf.keras.layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same',data_format='channels_last')(conv4_f)
    up7 = tf.keras.layers.concatenate([up7_1, conv3_f], axis=channel_axis)
    up7 = Rec_conv2d_bn(up7,128, 3, 3)
    conv7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same',data_format='channels_last')(up7)
    #conv7 = Conv2D(128, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv7)
    
    up8_1 =tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same',data_format='channels_last')(conv7)
    up8 = tf.keras.layers.concatenate([up8_1, conv2_f], axis=channel_axis)
    up8 = Rec_conv2d_bn(up8,128, 3, 3)
    conv8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_last')(up8)
    #conv8 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv8)
    
    up9_1 = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same',data_format='channels_last')(conv8)
    up9 = tf.keras.layers.concatenate([up9_1, conv1_f], axis=channel_axis)
    up9 = Rec_conv2d_bn(up9,128, 3, 3)
    conv9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(up9)
    #conv9 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv9)
    
   
    conv10 = tf.keras.layers.Conv2D(1,1, padding = "same", activation='sigmoid')(conv9)
       

    model = tf.keras.Model(inputs=[inputs], outputs=[conv10])

   #model.compile(optimizer=Adam(2e-4),loss=dice_coef_loss,metrics = [dice_coef, 'acc', 'mse'])
    
   #model.compile(optimizer=tf.train.AdamOptimizer(3e-4), loss='binary_crossentropy',metrics = ['acc', 'mse'])
    
    return model 


input_shape = (512, 512, 3)
model = build_R2UNetED(input_shape)
model.summary()
    ########################################

import os   
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
#from model import build_unet
#from metrics import dice_loss, dice_coef, iou

H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    set_seed = 42
    np.random.set_seed = set_seed
    

    """ Directory to save files """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 2
    lr = 1e-4

    num_epochs = 150
    model_path = os.path.join("files", "model.h5")
    csv_path = os.path.join("files", "data.csv")

    """ Dataset """
    dataset_path = "new_data"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "test")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_setps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_setps += 1

    """ Model """
    model = build_R2UNetED((H, W, 3))
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
    # model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_setps,
        callbacks=callbacks
    )
#from keras import Dropout
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
#from metrics import dice_loss, dice_coef, iou

H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

if __name__ == "__main__":
    """ Save the results in this folder """
    create_dir("results")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("files/model.h5")

    """ Load the dataset """
    dataset_path = os.path.join("new_data", "test")
    test_x, test_y = load_data(dataset_path)

    """ Make the prediction and calculate the metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extracting name """
        name = x.split("\\")[-1].split(".")[0] 

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype(np.int32)
        y_pred = np.squeeze(y_pred, axis=-1)

        """ Saving the images """
        save_image_path = f"results/{name}.png"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculate the metrics """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    """ Saving """
    df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 32) 9248        conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 512, 512, 32) 0           conv2d_1[0][0]                   
_______________________________________________________________________________________